In [ ]:
# @title #🛠️ 0. Prepare the environment

# Fake github identity for sync and clone
GITHUB_USERNAME = 'teto123'
GITHUB_EMAIL = 'teto@teto.teto'

# --- Setup python environtment ---

# Helper to print info
def info(msg):
    print(f"ℹ️  {msg}")

def success(msg):
    print(f"✅ {msg}")

def error(msg):
    print(f"❌ {msg}")

def get_device_tree_dir(content):
    match = re.search(r'LOCAL_PATH\s*:=\s*([^\s#]+)', content)
    return match.group(1).strip()
    if not match:
      raise Exception("LOCAL_PATH not found in device.mk")

def get_lunch_names(content):
    lunch_names = re.findall(r'\b(\w+(?:_\w+)*-(?:\w+(?:-\w+)*))\b', content)
    return lunch_names

# --- Install repo ---
!sudo mkdir -p /usr/local/bin
!sudo curl -o /usr/local/bin/repo https://storage.googleapis.com/git-repo-downloads/repo
!sudo chmod a+x /usr/local/bin/repo

# Verify
!repo --version

# --- Setup Orangefox ---
%cd ~
!sudo apt install git aria2 -y
!git clone https://gitlab.com/OrangeFox/misc/scripts
!cd scripts
!sudo bash setup/android_build_env.sh
!sudo bash setup/install_android_sdk.sh

# Set Git identity
!git config --global user.name {GITHUB_USERNAME}
!git config --global user.email {GITHUB_EMAIL}

success("Environment setup completed")

-------------

In [ ]:
# @title #🔄 1. Sync OrangeFox sources and minimal manifest
# @markdown > ⚠️ Android 14.1 needs more space than Colab has. It may fail.
# @markdown
# @markdown > 📝 You can build most Android 14 using 12.1 source. Just set SDK version to 31 or older in device.mk
import os
HOME = os.path.expanduser("~")

ORANGE_FOX_VERSION = '12.1' #@param [12.1, 14.1]
ORANGE_FOX_FOLDER = 'OrangeFox'
ORANGE_FOX_DIR = f"{HOME}/{ORANGE_FOX_FOLDER}"
!mkdir ~/OrangeFox_sync
%cd ~/OrangeFox_sync
!git clone https://gitlab.com/OrangeFox/sync.git # (or, using ssh, "git clone git@gitlab.com:OrangeFox/sync.git")
%cd ~/OrangeFox_sync/sync/
!yes | ./orangefox_sync.sh --branch {ORANGE_FOX_VERSION} --path {ORANGE_FOX_DIR}
%cd {ORANGE_FOX_DIR}
success(f"Sync done: {ORANGE_FOX_DIR}")

------------

In [ ]:
# @title 🌲 2. Add Device Tree to Source

# @markdown ## 🗂️ GitHub Device Tree URL
# @markdown URL of the device tree repository or file on GitHub.
GIT_DEVICE_TREE_URL = "https://github.com/chunix64/android_device_tecno_TECNO-CK6n/"  # @param {type: "string"}

#@markdown

# @markdown ## 🌿 GitHub Branch
# @markdown Branch to use when cloning the device tree.
GIT_DEVICE_TREE_BRANCH = "main"  # @param {type: "string"}

#@markdown

# @markdown ## 🧹 Delete Old Device Tree
# @markdown Replace device tree with the latest version.
DELETE_OLD_DEVICE_TREE = True  # @param {type: "boolean"}

#@markdown

%cd {ORANGE_FOX_DIR}

from urllib.parse import urlparse
import re
import requests
import os
import subprocess
import sys

try:
    # Parse GitHub URL
    url_parts = urlparse(GIT_DEVICE_TREE_URL).path.strip("/").split("/")
    if len(url_parts) < 2:
        raise ValueError("Invalid GitHub URL")

    github_username, github_repo = url_parts[:2]

    # Raw URLs
    device_tree_url = f"https://github.com/{github_username}/{github_repo}" # Fix user link
    device_mk_url = f"https://raw.githubusercontent.com/{github_username}/{github_repo}/{GIT_DEVICE_TREE_BRANCH}/device.mk"
    android_products_mk_url = f"https://raw.githubusercontent.com/{github_username}/{github_repo}/{GIT_DEVICE_TREE_BRANCH}/AndroidProducts.mk"

    # Fetch device.mk
    resp = requests.get(device_mk_url, timeout=10)
    if resp.status_code != 200:
        raise FileNotFoundError("device.mk not found. The GitHub repo is not a valid device tree.")

    device_tree_dir = get_device_tree_dir(resp.text)
    success(f"Device tree path: {device_tree_dir}")

except Exception as e:
    error(e)
    sys.exit(1)

# Clone device tree
clone_target_dir = f"{ORANGE_FOX_DIR}/{device_tree_dir}"
if os.path.exists(clone_target_dir) and DELETE_OLD_DEVICE_TREE:
    subprocess.run(f"rm -rf {clone_target_dir}", shell=True, check=False)
os.makedirs(clone_target_dir, exist_ok=True)

info(f"Cloning device tree...")
subprocess.run(
    f"git clone {device_tree_url} -b {GIT_DEVICE_TREE_BRANCH} {clone_target_dir}",
    shell=True,
    check=True
)
success("Device tree cloned!")

# Fetch AndroidProducts.mk
try:
    info("Fetching lunch names...")
    resp = requests.get(android_products_mk_url, timeout=10)
    if resp.status_code != 200:
        raise FileNotFoundError("AndroidProducts.mk not found. The GitHub repo is not a valid device tree.")

    lunch_names = get_lunch_names(resp.text)

    if lunch_names:
        success("Lunch names found:")
        print(" • " + "\n • ".join(lunch_names))
    else:
        info("No lunch names found.")

except Exception as e:
    error(e)


--------------------

In [ ]:
# @title 🤔 3. Check device trees avaible
%cd {ORANGE_FOX_DIR}
import os
devices_dir = os.path.join(ORANGE_FOX_DIR, "device")

def build_available_devices_table():
    table = {}
    for manufacturer in os.listdir(devices_dir):
        manufacturer_dir = os.path.join(devices_dir, manufacturer)
        if not os.path.isdir(manufacturer_dir):
            continue
        table[manufacturer] = {}
        for device in os.listdir(manufacturer_dir):
            device_dir = os.path.join(manufacturer_dir, device)
            if not os.path.isdir(device_dir):
                continue
            android_products_path = os.path.join(device_dir, "AndroidProducts.mk")
            if os.path.exists(android_products_path):
                content = open(android_products_path).read()
                device_lunch_names = get_lunch_names(content)
                # Filter only the lunch names you want
                device_lunch_names = [l for l in device_lunch_names if l in lunch_names]
                if device_lunch_names:
                    table[manufacturer][device] = device_lunch_names

        # Remove manufacturer if empty
        if not table[manufacturer]:
            del table[manufacturer]

    return table

devices_table = build_available_devices_table()


# Print table
from tabulate import tabulate

table_rows = []
for manufacturer, devices in devices_table.items():
    for device, lunch_names_list in devices.items():
        table_rows.append([manufacturer, device, "\n".join(lunch_names_list)])

print(tabulate(table_rows, headers=["Manufacturer", "Device", "Lunch Names"], tablefmt="fancy_grid"))


In [ ]:
# @title 🏗️ 4. Build OrangeFox
# @markdown ## 💿 Recovery Partition
# @markdown Choose the partition where the recovery ramdisk will be stored.
RECOVERY_IMAGE = "vendorbootimage" #@param ["vendorbootimage", "bootimage", "recoveryimage"]

#@markdown

# @markdown ## 🍽️ Device Lunch Name
# @markdown Use the device lunch name you found in the previous cell.
DEVICE_LUNCH_NAME = "twrp_CK6n-eng" #@param {type:"string"}

try:
    DEVICE_NAME = DEVICE_LUNCH_NAME.split('_')[1].split('-')[0]
except IndexError:
    DEVICE_NAME = ""

#@markdown

# Set linux environment variables
import os
os.environ["ALLOW_MISSING_DEPENDENCIES"] = "true"
os.environ["FOX_BUILD_DEVICE"] = DEVICE_NAME
os.environ["LC_ALL"] = "C"

# Build command
build_cmd = f"""
source {ORANGE_FOX_DIR}/build/envsetup.sh
lunch {DEVICE_LUNCH_NAME}
mka adbd {RECOVERY_IMAGE}
"""

# Run in shell
!bash -c "{build_cmd}"

--------